# Synthetic example

In this notebook, we show the usage the new implementaiton of the IPA method with the simulated dataset dataset introduced with the previous version of the method ([Del Carratore et al., 2019](https://pubs.acs.org/doi/full/10.1021/acs.analchem.9b02354)).
The synthetic experiment was built by considering 15 compounds involved in the mevalonate pathway and limonene synthesis (compounds highlighted in blue in the figure below).
![Mevalonate](ExampleDatasets/Synthetic/mevalonate_pathway.jpeg)
Several Adducts and isotopes were simulated for each of the considered metabolites, both in negative and positive mode, consistently with the relative concentrations shown int he table below. A detailed description of how the simulated datasets can be found in the original paper ([Del Carratore et al., 2019](https://pubs.acs.org/doi/full/10.1021/acs.analchem.9b02354)).
 


In [1]:
import pandas as pd

compounds = pd.read_csv('ExampleDatasets/Synthetic/synthetic_compounds_info.csv')
compounds

,id,name,Formula,RT,Relative Concentration
0,C00024,Acetyl-CoA,C23H38N7O17P3S1,60,19.1
1,C00332,Acetoacetyl-CoA,C25H40N7O18P3S,75,95.7
2,C00010,Coenzyme A,C21H36N7O16P3S,300,24.9
3,C00356,3-Hydroxy-3-methylglutaryl-CoA,C27H44N7O20P3S,120,36.8
4,C00005,NADPH,C21H30N7O17P3,200,76.1
5,C00006,NADP,C21H29N7O17P3,150,77.8
6,C00418,Mevalonic acid,C6H12O4,250,34.9
7,C00002,ATP,C10H16N5O13P3,210,6.7
8,C00008,ADP,C10H15N5O10P2,211,9.8
9,C01107,Mevalonic acid-5P,C6H13O7P,310,35.8


## Positive mode
The dataset generated in positive mode contains 83 mass spectrometry features and it can be found in the .csv file shown below.

In [2]:
df_all = pd.read_csv('ExampleDatasets/Synthetic/positive_synth_dataset.csv')
df_all.head()

,ids,rel.ids,mzs,RTs,Int,Compound ID,Formula,Adduct,isotope,charge
0,1,0,810.133740,60.534708,19.100000,C00024,C23H39N7O17P3S1,M+H,mono,1
1,2,0,832.115830,61.629705,9.000503,C00024,C23H38N7O17P3S1Na1,M+Na,mono,1
2,3,0,405.571155,60.727202,16.601507,C00024,C23H40N7O17P3S1,M+2H,mono,2
3,4,0,1619.258146,60.988866,9.287749,C00024,C46H77N14O34P6S2,2M+H,mono,1
4,5,1,852.144271,75.607828,95.700000,C00332,C25H41N7O18P3S1,M+H,mono,1


The .csv file also containes the correct annotation for each feature. To use the IPA method, a dataframe only containg the necessary information is required.

In [3]:
df = df_all.copy()
df=df.drop(['Compound ID', 'Formula','Adduct','isotope','charge'], axis=1)
df.head()

,ids,rel.ids,mzs,RTs,Int
0,1,0,810.133740,60.534708,19.100000
1,2,0,832.115830,61.629705,9.000503
2,3,0,405.571155,60.727202,16.601507
3,4,0,1619.258146,60.988866,9.287749
4,5,1,852.144271,75.607828,95.700000


We also need to import the ipaPy2 module and the necessary database.

In [4]:
from ipaPy2 import ipa
DB=pd.read_csv('DB/IPA_MS1.csv')
adducts = pd.read_csv('DB/adducts.csv')

At this point, the whole IPA pipeline can be run with the simpleIPA() function.
It should be noticed that in this notebook 5 cores are used for the analysis. The user should be very carefull with the ncores parameter, which should be chosen considering the hardware used.

In [5]:
annotations = ipa.simpleIPA(df=df,ionisation=1,DB=DB,adductsAll=adducts,ppm=15,
                            delta_add=0.1,delta_bio=0.1,burn=1000,noits=5000,ncores=5)

mapping isotope patterns ....
0.2 seconds elapsed
computing all adducts - Parallelized ....
35.0 seconds elapsed
annotating based on MS1 information - Parallelized ...
4.1 seconds elapsed
computing all possible biochemical connections - Parallelized
considering the reactions stored in the database ...
11.9 seconds elapsed
computing posterior probabilities including biochemical and adducts connections
initialising sampler ...


Gibbs Sampler Progress Bar: 100%|██████████| 5000/5000 [02:48<00:00, 29.60it/s]


parsing results ...
Done -  169.1 seconds elapsed


Similarly to what shown in the original paper, having the correct annotations for each mass spectrometry feature, we can use the Logarithmic Predictive Score (LPS) to evaluate the performance of the method.
The LPS score is computed as follows:

$LPS = \sum \limits _{i=0} ^{M} p_{i} $

where $p_i$ is the probability assigned to the correct annotation for the $i^{th}$ feature.
In the best case scenario the LPS score is euqual to zero. In all other cases LPS is a negative value.

In the code below, the prior and posterior proabilities assigned to the correct annotations are extracted from the IPA output.

In [6]:
prior = []
post = []
postGibbs = []
p_Chisq = []
for k in range(0,len(df_all.index)):
    key = df_all['ids'][k]
    if(key in list(annotations.keys())):        
        tmp = annotations[key]
        GT = df_all['Compound ID'][k]
        prior.append(list(tmp[tmp['id']==GT]['prior'])[0])
        post.append(list(tmp[tmp['id']==GT]['post'])[0])
        postGibbs.append(list(tmp[tmp['id']==GT]['post Gibbs'])[0])
        p_Chisq.append(list(tmp[tmp['id']==GT]['chi-square pval'])[0])
    else:
        prior.append(None)
        post.append(None)
        postGibbs.append(None)
        p_Chisq.append(None)

df_res= df_all.copy()
df_res['prior'] = prior
df_res['post'] = post
df_res['post Gibbs'] = postGibbs
df_res['chi-square pval'] = p_Chisq

df_res=df_res[df_res['isotope']=='mono']
df_res=df_res.drop(['isotope', 'charge'], axis=1)

The LPS score associated with the prior probability is computed below:

In [7]:
import numpy as np
np.sum(np.log10(df_res['prior']))

-41.17109981829767

The LPS score associated with the posterior probability, shown below, is significantly higher.

In [8]:
np.sum(np.log10(df_res['post Gibbs']))

-31.87944546560881

It must be mentioned that this scores are not directly comparable with the ones computed with the old implementation of the IPA method.

# Negative mode
The dataset generated in negative mode contains 95 mass spectrometry features and it can be found in the .csv file shown below.

In [9]:
df_all = pd.read_csv('ExampleDatasets/Synthetic/negative_synth_dataset.csv')
df_all.head()

,ids,rel.ids,mzs,RTs,Int,Compound ID,Formula,Adduct,isotope,charge
0,1,0,808.117931,60.168864,15.454966,C00024,C23H37N7O17P3S1,M-H,mono,-1
1,2,0,1617.244072,59.832483,16.612324,C00024,C46H75N14O34P6S2,2M-H,mono,-1
2,3,0,1618.246019,61.852075,8.265022,C00024,C45[13]C1H75N14O34P6S2,2M-H,iso,-1
3,4,0,403.554946,60.069536,5.311077,C00024,C23H36N7O17P3S1,M-2H,mono,-2
4,5,0,2426.368968,58.812628,13.085050,C00024,C69H113N21O51P9S3,3M-H,mono,-1


Repeating the same steps done for the positive dataset, we can run the IPA method.

In [10]:
df = df_all.copy()
df=df.drop(['Compound ID', 'Formula','Adduct','isotope','charge'], axis=1)

In [11]:
annotations = ipa.simpleIPA(df=df,ionisation=-1,DB=DB,adductsAll=adducts,ppm=15,
                            delta_add=0.1,delta_bio=0.1,burn=1000,noits=5000,ncores=5)

mapping isotope patterns ....
0.2 seconds elapsed
computing all adducts - Parallelized ....
154.2 seconds elapsed
annotating based on MS1 information - Parallelized ...
4.8 seconds elapsed
computing all possible biochemical connections - Parallelized
considering the reactions stored in the database ...
13.4 seconds elapsed
computing posterior probabilities including biochemical and adducts connections
initialising sampler ...


Gibbs Sampler Progress Bar: 100%|██████████| 5000/5000 [02:52<00:00, 28.98it/s]


parsing results ...
Done -  172.7 seconds elapsed


Also in this case we can compute the LPS scores

In [12]:
prior = []
post = []
postGibbs = []
p_Chisq = []
for k in range(0,len(df_all.index)):
    key = df_all['ids'][k]
    if(key in list(annotations.keys())):        
        tmp = annotations[key]
        GT = df_all['Compound ID'][k]
        prior.append(list(tmp[tmp['id']==GT]['prior'])[0])
        post.append(list(tmp[tmp['id']==GT]['post'])[0])
        postGibbs.append(list(tmp[tmp['id']==GT]['post Gibbs'])[0])
        p_Chisq.append(list(tmp[tmp['id']==GT]['chi-square pval'])[0])
    else:
        prior.append(None)
        post.append(None)
        postGibbs.append(None)
        p_Chisq.append(None)

df_res= df_all.copy()
df_res['prior'] = prior
df_res['post'] = post
df_res['post Gibbs'] = postGibbs
df_res['chi-square pval'] = p_Chisq

df_res=df_res[df_res['isotope']=='mono']
df_res=df_res.drop(['isotope', 'charge'], axis=1)

In [13]:
np.sum(np.log10(df_res['prior']))

-40.15436067911171

In [14]:
np.sum(np.log10(df_res['post Gibbs']))

-32.04465455654571